In [1]:
if 'google.colab' in str(get_ipython()):
    # Colab specific setup
    assignment_path = '/content/gdrive/My Drive/CIS680/FinalProject/'
    
    # Mount your drive
    from google.colab import drive
    drive.mount("/content/gdrive")
    
    # Setup assignment folder and switch
    import os
    os.makedirs(assignment_path, exist_ok=True)
    os.chdir(assignment_path)

Mounted at /content/gdrive


Imports

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
from scipy import spatial
import cv2

# Get center of mass for RLE encoded mask

In [ ]:
import pycocotools.mask as mask
from shapely.geometry import Polygon
import pycocotools.mask as mask

def polygonCOMFromMask(compactRLESegmentation):
  maskedArr = mask.decode(compactRLESegmentation)
  area = float((maskedArr > 0.0).sum())
  # adapted from https://github.com/hazirbas/coco-json-converter/blob/master/generate_coco_json.py
  contours, _ = cv2.findContours(maskedArr, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
  segmentation = []
  valid_poly = 0
  for contour in contours:
  # Valid polygons have >= 6 coordinates (3 points)
     if contour.size >= 6:
        segmentation.append(contour.astype(float).flatten().tolist())
        valid_poly += 1
  if valid_poly == 0:
     return -1
  contour = np.array(segmentation[0])
  contour = Polygon(contour.reshape((-1, 2)))
  com = list(list(contour.centroid.coords)[0])
  return com

# Convert annotations in the desired point annotation and boxes format

In [ ]:
import json
with open("annotations.json") as f:
  data = json.load(f)
annotations = {}
id2file = {}
for image in data["images"]:
  id2file[image["id"]] = image["file_name"]
  annotations[image["file_name"]] = {"box_examples_coordinates": [], "points": []}
for annotation in data["annotations"]:
  x1, y1, x2, y2 = annotation["bbox"][0], annotation["bbox"][1], annotation["bbox"][0]+annotation["bbox"][2], annotation["bbox"][1]+annotation["bbox"][3]
  boxes = [[x1, y1], [x1, y2], [x2, y1], [x2, y2]]
  annotations[id2file[annotation["image_id"]]]["box_examples_coordinates"].append(boxes)
  com = polygonCOMFromMask(annotation["segmentation"])
  if com != -1:
    annotations[id2file[annotation["image_id"]]]["points"].append(com)

In [ ]:
with open("parts_new_annotations_v2.json", "w") as f:
  json.dump(annotations, f)

# Generate Density Maps

In [18]:
def matlab_style_gauss2D(shape=(3,3),sigma=0.5):
    """
    2D gaussian mask - should give the same result as MATLAB's
    fspecial('gaussian',[shape],[sigma])
    http://stackoverflow.com/questions/17190649/how-to-obtain-a-gaussian-filter-in-python
    """
    m,n = [(ss-1.)/2. for ss in shape]
    y,x = np.ogrid[-m:m+1,-n:n+1]
    h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
    h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
    sumh = h.sum()
    if sumh != 0:
        h /= sumh
    return h

def generate_density_map(points, filename):
  points = np.array(points).astype(int)
  tree = scipy.spatial.KDTree(points.copy(), leafsize=10)
  dists, neighbours = tree.query(points, k = 2)
  avg = np.average(dists[:, 1])

  test = np.zeros((384, 384))
  for i in range(points.shape[0]):
      y = points[i, 1]
      x = points[i, 0]
      test[y, x] = 1

  filt = matlab_style_gauss2D((avg, avg), avg/4)

  gt_generated = cv2.filter2D(test.copy(), -1, filt, 0)
  # print(gt_generated.sum())
  # plt.imshow(gt_generated)
  with open("parts_gt_density_maps/"+filename[:-4]+".npy", "wb") as f:
    np.save(f, gt_generated)

In [ ]:
with open("data/parts_new_annotations_scaled.json") as f:
  data = json.load(f)
for i, annotation in enumerate(data):
  print(i)
  generate_density_map(data[annotation]["points"], annotation)

# Scale Images

In [ ]:
for img_name in os.listdir("parts_dataset/"):
  print(img_name)
  image = cv2.imread("parts_dataset/"+img_name)
  image = cv2.resize(image, (384, 384))
  cv2.imwrite("parts_dataset_resized/"+img_name, image)

# Scale Annotations

In [ ]:
import json
import numpy as np
with open("parts_new_annotations_v2.json") as f:
  data = json.load(f)
for annotation in data:
  boxes =  data[annotation]["box_examples_coordinates"]
  points = data[annotation]["points"]
  boxes = np.array(boxes) * (384/1080)
  boxes = boxes.astype(int)
  points = np.array(points) * (384/1080)
  boxes, points = boxes.tolist(), points.tolist()
  data[annotation]["box_examples_coordinates"] = boxes
  data[annotation]["points"] = points
with open("parts_new_annotations_scaled.json", "w") as f:
  json.dump(data, f)

# Generate dataset split

In [ ]:
import random
images = os.listdir("parts_dataset_resized/")
random.shuffle(images)
train = images[:6480]
val = images[6480:7290]
test = images[7290:]
data = {"train": train, "val": val, "test": test}
with open("dataset_split.json", "w") as f:
  json.dump(data, f)

In [ ]:
import random
images = os.listdir("parts_dataset_resized/")
with open("image_classes.txt", "w") as f:
  for image in images:
    f.write(image+" machine-parts\n")

# Install dependencies and clone repo

In [ ]:
!pip install matplotlib opencv-python notebook tqdm
!pip install torch==1.4.0 torchvision==0.5.0

In [ ]:
!git clone https://github.com/cvlab-stonybrook/LearningToCountEverything

Cloning into 'LearningToCountEverything'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 93 (delta 30), reused 42 (delta 27), pack-reused 43
Unpacking objects: 100% (93/93), done.


In [23]:
from PIL import Image
import json
gt_dir = "parts_gt_density_maps"
anno_file = "data/parts_new_annotations.json"
im_id = "899_angle3_img.png"
with open(anno_file) as f:
    annotations = json.load(f)
anno = annotations[im_id]
bboxes = anno['box_examples_coordinates']
dots = np.array(anno['points'])
print(len(dots))

density_path = gt_dir + '/' + im_id.split(".png")[0] + ".npy"
density = np.load(density_path).astype('float32')    
print(density.sum())

25
25.0


# Training

In [ ]:
!python LearningToCountEverything/train.py

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100% 97.8M/97.8M [00:01<00:00, 95.2MB/s]
Training on train set data
  0% 0/6480 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
actual-predicted:   20.0,   22.7, error:    2.7. Current MAE:  2.73, RMSE:  2.73 Best VAL MAE: 10000000.00, RMSE: 10000000.00:   0% 0/6480 [00:00<?, ?it/s]
actual-predicted:   22.0,   22.6, error:    0.6. Current MAE:  1.68, RMSE:  1.98 Best VAL MAE: 10000000.00, RMSE: 10000000.00:   0% 1/6480 [00:01<1:45:56,  1.02it/s]
actual-predicted:   48.0,   30.2, error:   17.8. Current MAE:  7.06, RMSE: 10.42 Best VAL M